<a href="https://colab.research.google.com/github/lipeng2021/-python1/blob/main/5_7vgg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from IPython import display
import torchvision
import torchvision.transforms as transforms
device =torch.device('cuda'if torch.cuda.is_available()else 'cpu')
FILENAME = '/home.lp'
import time

In [2]:
class FlattenLayer(nn.Module):
  def __init__(self):
    super(FlattenLayer,self).__init__()
  def forward(self,x):
    y = x.view(x.shape[0],-1)
    return y

In [3]:
def vgg_block(num_covs,in_channels,out_channels):
  blk = []
  for i in range(num_covs):
    if i==0:
      blk.append(nn.Conv2d(in_channels,out_channels,kernel_size=3,padding=1))
    else:
      blk.append(nn.Conv2d(out_channels,out_channels,kernel_size=3,padding=1))
    blk.append(nn.ReLU())
  blk.append(nn.MaxPool2d(kernel_size=2,stride=2))
  return nn.Sequential(*blk)

In [4]:
conv_arch = ((1,1,64),(1,64,128),(2,128,256),(2,256,512),(2,512,512))
fc_features = 512*7*7
fc_hidden_units = 4096

In [5]:
def vgg(conv_arch,fc_features,fc_hidden_units=4096):
  net = nn.Sequential()
  for i,(num_convs,in_channels,out_channels) in enumerate(conv_arch):
    net.add_module("vgg_block_"+str(i+1),vgg_block(num_convs,in_channels,out_channels))
  net.add_module("fc",nn.Sequential(FlattenLayer(),
                nn.Linear(fc_features,fc_hidden_units),
                nn.ReLU(),
                nn.Dropout(0.5),
                nn.Linear(fc_hidden_units,fc_hidden_units),
                nn.ReLU(),
                nn.Dropout(0.5),
                nn.Linear(fc_hidden_units,10)
                  ))
  return net

In [6]:
net = vgg(conv_arch,fc_features,fc_hidden_units)
x = torch.rand(1,1,224,224)
for name,blk in net.named_children():
  x = blk(x)
  print(name,'output shape:',x.shape)

vgg_block_1 output shape: torch.Size([1, 64, 112, 112])
vgg_block_2 output shape: torch.Size([1, 128, 56, 56])
vgg_block_3 output shape: torch.Size([1, 256, 28, 28])
vgg_block_4 output shape: torch.Size([1, 512, 14, 14])
vgg_block_5 output shape: torch.Size([1, 512, 7, 7])
fc output shape: torch.Size([1, 10])


In [7]:
ratio =8
small_conv_arch = conv_arch = [(1,1,64//ratio),(1,64//ratio,128//ratio),(2,128//ratio,256//ratio),(2,256//ratio,512//ratio),(2,512//ratio,512//ratio)]
net = vgg(small_conv_arch,fc_features//ratio,fc_hidden_units//ratio)
print(net)

Sequential(
  (vgg_block_1): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (vgg_block_2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (vgg_block_3): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (vgg_block_4): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [8]:
def load_data_fashion_mnist(batch_size,resize=None,root=FILENAME):
  trans =[]
  if resize:
    trans.append(transforms.Resize(size=resize))
  trans.append(transforms.ToTensor())
  transform = transforms.Compose(trans)
  mnist_train = torchvision.datasets.FashionMNIST(root=root,train=True,transform=transform,download=True)
  mnist_test = torchvision.datasets.FashionMNIST(root=root,train=False,transform=transform,download=True)
  train_iter = torch.utils.data.DataLoader(mnist_train,batch_size,shuffle=True,num_workers=2)
  test_iter = torch.utils.data.DataLoader(mnist_test,batch_size,shuffle=True,num_workers=2)
  return train_iter,test_iter
batch_size = 64
train_iter,test_iter = load_data_fashion_mnist(batch_size,resize=224)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting /home.lp/FashionMNIST/raw/train-images-idx3-ubyte.gz to /home.lp/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting /home.lp/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /home.lp/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting /home.lp/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /home.lp/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting /home.lp/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /home.lp/FashionMNIST/raw



In [9]:
def evaculate_accuracy(data_iter,net,device):
  acc_sum = 0.0 
  n = 0
  with torch.no_grad():
    for X,y in data_iter:
      if isinstance(net,nn.Module):
        net.eval()
        acc_sum += (net(X.to(device)).argmax(dim=1)==y.to(device)).float().sum().cpu().item()
        net.train()
      else:
        if ('is_training'in net.__code__.co_varnames):
          acc_sum += (net(X,is_training=False).argmax(dim=1)==y).float().sum().item()
        else:
          acc_sum += (net(X).argmax(dim=1)==y).float().sum().item()
      n += y.shape[0]
    return acc_sum/n

In [14]:
def train_ch5(net,train_iter,test_iter,device,batch_size,optimizer,num_epochs):
  loss = torch.nn.CrossEntropyLoss()
  net.to(device)
  print('train on:',device)
  for epoch in range(num_epochs):
    train_loss=0.0
    train_acc_sum =0.0
    n =0
    batch_cout = 0
    start = time.time()
    for X,y in train_iter:
      X = X.to(device)
      y = y.to(device)
      y_hat = net(X)
      l = loss(y_hat,y)
      optimizer.zero_grad()
      l.backward()
      optimizer.step()
      train_loss += l.cpu().item()
      train_acc_sum += (net(X).argmax(dim=1)==y).float().sum().cpu().item()
      n += y.shape[0]
      batch_cout += 1
    print(batch_cout)
    test_acc_sum = evaculate_accuracy(test_iter,net,device)
    print('epoch:%d,loss:%.4f,train_acc:%.3f,test_acc:%.3f,time:%.1f'%(epoch+1,train_loss/batch_cout,train_acc_sum/n,test_acc_sum,time.time()-start))
 

In [15]:
lr=0.001
num_epochs =5
optimizer = torch.optim.Adam(net.parameters(),lr=lr)
train_ch5(net,train_iter,test_iter,device,batch_size,optimizer,num_epochs)

train on: cuda
938
epoch:1,loss:0.1944,train_acc:0.936,test_acc:0.915,time:130.0
938
epoch:2,loss:0.1794,train_acc:0.941,test_acc:0.927,time:129.6
938
epoch:3,loss:0.1609,train_acc:0.948,test_acc:0.927,time:129.5
938
epoch:4,loss:0.1517,train_acc:0.951,test_acc:0.925,time:129.2
938
epoch:5,loss:0.1391,train_acc:0.955,test_acc:0.926,time:129.7
